<a href="https://colab.research.google.com/github/ThisIsNSK/BU-DCA-DeepLearning/blob/main/Framework_Test_CatsVsDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
DIR_PATH = "/content/drive/MyDrive/deep_learning/CatsVsDogs_Dataset"

In [ ]:
import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import matplotlib.pyplot as plt
import numpy as np

import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
ROOT_DIR = "/content/gdrive/MyDrive/CatsVsDogs_Dataset/"

train_dir = ROOT_DIR + "train"
validation_dir = ROOT_DIR + "validation"

## Train
train_class1_dir = ROOT_DIR + "train/Cat"
train_class2_dir = ROOT_DIR + "train/Dog"


## Validation
validation_class1_dir = ROOT_DIR + "validation/Cat"
validation_class2_dir = ROOT_DIR + "validation/Dog"



In [ ]:
num_class1_tr = len(os.listdir(train_class1_dir))
num_class2_tr = len(os.listdir(train_class2_dir))


num_class1_val = len(os.listdir(validation_class1_dir))
num_class2_val = len(os.listdir(validation_class2_dir))


total_train = num_class1_tr + num_class2_tr 
total_val = num_class1_val + num_class2_val

print('Total training Class-1 images:', num_class1_tr)
print('Total training Class-2 images:', num_class2_tr)


print("--")
print('Total validation Class-1 images:', num_class1_val)
print('Total validation Class-2 images:', num_class2_val)


print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)



FileNotFoundError: ignored

In [ ]:
BATCH_SIZE = 10  # Number of training examples to process before updating our models variables
IMG_SHAPE  = 100  # Our training data consists of images with width of 150 pixels and height of 150 pixels

In [ ]:
train_image_generator      = ImageDataGenerator(rescale=1./255)  # Generator for our training data
validation_image_generator = ImageDataGenerator(rescale=1./255)  # Generator for our validation data
'''
, featurewise_std_normalization=True,
samplewise_std_normalization=True,
horizontal_flip=True,
vertical_flip=True,
#brightness_range=(0.0,0.25),
rotation_range=45)  # Generator for our validation data

test_image_generator_1     = ImageDataGenerator(rescale=1./255, featurewise_std_normalization=True,
samplewise_std_normalization=True,
horizontal_flip=True,
vertical_flip=True,
#brightness_range=(0.0,0.25),
rotation_range=45)  # Generator for our validation data
'''



"""After defining our generators for training and validation images, **flow_from_directory** method will load images from the disk, apply rescaling, and resize them using single line of code."""

train_data_gen = train_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                           directory=train_dir,
                                                           shuffle=True,
                                                           target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                           class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                              directory=validation_dir,
                                                              shuffle=True,
                                                              target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                              class_mode='binary')

#test_data_gen = test_image_generator.flow_from_directory(batch_size=BATCH_SIZE,
                                                         #directory=test_dir,
                                                         #shuffle=True,
                                                         #target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                         #class_mode='binary')
#test_data_gen_1 = test_image_generator_1.flow_from_directory(batch_size=BATCH_SIZE,
                                                         #directory=test_dir_1,
                                                         #shuffle=True,
                                                         #target_size=(IMG_SHAPE,IMG_SHAPE), #(150,150)
                                                         #class_mode='binary')


In [ ]:
sample_training_images, _ = next(train_data_gen)

"""The `next` function returns a batch from the dataset. One batch is a tuple of (*many images*, *many labels*). For right now, we're discarding the labels because we just want to look at the images."""

# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

plotImages(sample_training_images[:5])  # Plot images 0-4

NameError: ignored

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(IMG_SHAPE, IMG_SHAPE, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),  ### 128 -> 256??
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(512, (3,3), activation='relu'),  ### 128 -> 256??
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
EPOCHS = 20
#callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
history = model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=EPOCHS,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))),
    #callbacks=[callback]
)

In [ ]:


acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(EPOCHS)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig('./foo.png')
plt.show()

In [ ]:
## Getting the True Labels

val_image_batch, val_label_batch = next(iter(val_data_gen))
true_label_ids = np.argmax(val_label_batch, axis=-1)
#print("Validation batch shape:", val_image_batch.shape)
true_label_ids
val_label_batch[0]
#true_label_ids.shape

In [ ]:

## Prediction - Validation & Result Ploting

tf_model_predictions = model.predict(val_image_batch)
print("Prediction results shape:", tf_model_predictions.shape)
plt.figure(figsize=(10,9))
plt.subplots_adjust(hspace=0.5)
for n in range((len(tf_model_predictions))):
  plt.subplot(7,5,n+1)
  plt.imshow(val_image_batch[n])
  color = "green" if np.argmax(tf_model_predictions[n]) == val_label_batch[n] else "red"
  plt.title(str(np.argmax(tf_model_predictions[n])) + "/" + str(val_label_batch[n]), color=color)
  plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect) \n Label: Predicted/Actual")

In [ ]:
## Getting the True Labels

val_image_batch, val_label_batch = next(iter(val_data_gen))
true_label_ids = np.argmax(val_label_batch, axis=-1)
#print("Validation batch shape:", val_image_batch.shape)
true_label_ids
val_label_batch
#true_label_ids.shape

In [ ]:

## Prediction - Validation & Result Ploting
for i in range (total_val//BATCH_SIZE):
  val_image_batch, val_label_batch = next(iter(val_data_gen))
  tf_model_predictions = model.predict(val_image_batch)
  print("Prediction results shape:", tf_model_predictions.shape)
  plt.figure(figsize=(10,9))
  plt.subplots_adjust(hspace=0.5)
  for n in range((len(tf_model_predictions))):
    plt.subplot(7,5,n+1)
    plt.imshow(val_image_batch[n])
    color = "green" if np.argmax(tf_model_predictions[n]) == val_label_batch[n] else "red"
    plt.title(str(np.argmax(tf_model_predictions[n])) + "/" + str(int(val_label_batch[n])), color=color)
    plt.axis('off')
_ = plt.suptitle("Model predictions (green: correct, red: incorrect) \n Label: Predicted/Actual")